In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"

from src.pipeline import Config, build_xy, FlagsDataset, CNN1D, LABELS

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
cfg = Config()
np.random.seed(cfg.seed)
torch.manual_seed(cfg.seed)

CSV_PATH = DATA_DIR / "EURUSD_15m_007.csv"
JSON_PATH = DATA_DIR / "EURUSD_15m_007_cimkezett.json"

X, y = build_xy(CSV_PATH, JSON_PATH, L=cfg.L, pre_bars=cfg.pre_bars)
print("Dataset:", X.shape, y.shape)

batch_size = 16
idx = np.random.RandomState(cfg.seed).choice(len(y), size=min(batch_size, len(y)), replace=False)

X_small = X[idx]
y_small = y[idx]

ds = FlagsDataset(X_small, y_small)
loader = DataLoader(ds, batch_size=len(ds), shuffle=True)  

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN1D(in_ch=X.shape[1], n_classes=len(LABELS)).to(device)

loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=3e-3, weight_decay=0.0)

epochs = 400

for ep in range(1, epochs + 1):
    model.train()
    xb, yb = next(iter(loader))
    xb, yb = xb.to(device), yb.to(device)

    opt.zero_grad(set_to_none=True)
    logits = model(xb)
    loss = loss_fn(logits, yb)
    loss.backward()
    opt.step()

    pred = logits.argmax(dim=1)
    acc = (pred == yb).float().mean().item()

    if ep % 25 == 0 or ep == 1:
        print(f"ep={ep:03d}  loss={loss.item():.6f}  acc={acc:.3f}")

    if acc >= 1.0 and loss.item() < 0.01:
        print("Overfit success (single batch).")
        break

model.eval()
with torch.no_grad():
    logits = model(xb)
    pred = logits.argmax(dim=1).cpu().numpy()
print("true:", y_small)
print("pred:", pred)

Dataset: (100, 4, 256) (100,)
ep=001  loss=2.054248  acc=0.000
ep=025  loss=0.088358  acc=1.000
Overfit success (single batch).
true: [1 2 0 1 3 5 5 1 3 0 1 0 1 3 5 0]
pred: [2 3 5 1 3 5 1 3 0 1 0 0 1 5 1 0]
